In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
!git clone git@github.com:swaggy66/M-ABSA.git

Cloning into 'M-ABSA'...


In [41]:
import os
import pandas as pd
import datasets

from glob import glob
from tqdm.auto import tqdm
from konoise import NoiseGenerator

In [108]:
genertor = NoiseGenerator()
noise_methods = ['change-vowels', 'palatalization', 'linking', 'liquidization', 'nasalization', 'assimilation']
categories = []
splits = []
sentences = []
sentences_noisy = []
noise_method_selected = []
for path in tqdm(glob('M-ABSA/data/*/ko/*.txt')):
    # 경로를 os.path.normpath로 정리하고, 폴더 이름 분리
    parts = os.path.normpath(path).split(os.sep)
    category = parts[2]  # 3번째 폴더
    split = os.path.splitext(parts[-1])[0]  # 파일명 (확장자 제외)
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for sentence in tqdm(lines):
            sentence = sentence.strip()
            sentence = sentence.split('####')[0]
            if split != 'test':
                # sentence_noisy = genertor.generate(sentence, ','.join(noise_methods), 1)
                # categories.append(category)
                # splits.append(split)
                # sentences.append(sentence)   
                # sentences_noisy.append(sentence_noisy[0][0])
                pass
            else:
                for noise_method in noise_methods:
                    sentence_noisy = genertor.generate(sentence, noise_method, 1)[0][0]
                    if sentence == sentence_noisy:
                        continue
                    categories.append(category)
                    splits.append(split)
                    sentences.append(sentence)   
                    sentences_noisy.append(sentence_noisy)
                    noise_method_selected.append(noise_method)
                
                sentence_noisy_all = sentence
                for noise_method in noise_methods:
                    sentence_noisy_all = genertor.generate(sentence_noisy_all, noise_method, 1)[0][0]
                if sentence == sentence_noisy_all:
                    continue
                categories.append(category)
                splits.append(split)
                sentences.append(sentence)   
                sentences_noisy.append(sentence_noisy_all)
                noise_method_selected.append('all')

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/566 [00:00<?, ?it/s]

  0%|          | 0/1278 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

  0%|          | 0/544 [00:00<?, ?it/s]

  0%|          | 0/1264 [00:00<?, ?it/s]

  0%|          | 0/281 [00:00<?, ?it/s]

  0%|          | 0/524 [00:00<?, ?it/s]

  0%|          | 0/1181 [00:00<?, ?it/s]

In [109]:
df = pd.DataFrame({
    'category': categories,
    'split': splits,
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'noise_method': noise_method_selected
})

In [110]:
test_df = df[df['split'] == 'test'].drop(columns=['split']).reset_index(drop=True)

In [111]:
groupby = test_df.groupby(['sentence'])[['noise_method']].count().reset_index()

In [112]:
not_all_sentences = groupby[groupby['noise_method']==2]['sentence'].tolist()

In [115]:
cond1 = test_df['noise_method']=='all'

In [116]:
cond2 = test_df['sentence'].isin(not_all_sentences)

In [122]:
test_df = test_df[(cond1 & cond2)==False].reset_index(drop=True)

In [124]:

test_ds = datasets.Dataset.from_pandas(test_df)

In [125]:
ds = datasets.DatasetDict({
    'test': test_ds
})

In [126]:
ds.push_to_hub(repo_id=f'jwengr/M-ABSA-ko-noise')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/395 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jwengr/M-ABSA-ko-noise/commit/52e0b2abb88eda82356fd4c49c62cbf5d6c8494c', commit_message='Upload dataset', commit_description='', oid='52e0b2abb88eda82356fd4c49c62cbf5d6c8494c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jwengr/M-ABSA-ko-noise', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jwengr/M-ABSA-ko-noise'), pr_revision=None, pr_num=None)